In [17]:
import torch
from torch2trt import TRTModule
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
import time

DO_DISPLAY_CAMERA = True
DO_RECORD_CAMERA = False
DO_MOTOR = True

RTR_FILENAME = '../models/resnet18_0.0080_trt.pth'
# RTR_FILENAME = '../models/resnet34_0.0069_trt.pth'
# RTR_FILENAME = '../models/resnet50_0.0044_trt.pth'

In [18]:
%%time

device = torch.device('cuda')
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(RTR_FILENAME))

CPU times: user 1.62 s, sys: 508 ms, total: 2.12 s
Wall time: 3.49 s


<All keys matched successfully>

In [21]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

camera = Camera()
if DO_DISPLAY_CAMERA:
    image_widget = ipywidgets.Image()
    # traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
    display(image_widget)

if DO_MOTOR:
    from jetbot import Robot
    robot = Robot()

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.3, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
display(ipywidgets.HBox([ipywidgets.VBox([speed_gain_slider, steering_gain_slider,
                                         steering_dgain_slider, steering_bias_slider]),
                         y_slider, speed_slider,
                         ipywidgets.VBox([x_slider, steering_slider])]))

angle = 0.0
angle_last = 0.0

if DO_RECORD_CAMERA:
    record_camera = cv2.VideoWriter('result.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10.0, (320, 480))

def execute(change):
    global angle, angle_last
    start = time.time()
    image = change['new']
    
    if DO_DISPLAY_CAMERA:
        image_widget.value = cv2.imencode('.jpg', image)[1].tobytes()
    
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    if DO_RECORD_CAMERA:
        cv2.putText(image, 'X: {:.2f}'.format(x), (10, 200), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, 'Y: {:.2f}'.format(y), (10, 250), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
    
#     print('\r', x, y, end='')
    x_slider.value = x
    y_slider.value = y
    speed_slider.value = speed_gain_slider.value
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    if DO_MOTOR:
        left_motor = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        right_motor = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
#         robot.left_motor.value = left_motor
#         robot.right_motor.value = right_motor
#         print('motor: {:.2f} {:.2f}'.format(left_motor, right_motor), end='\n')
        
        if DO_RECORD_CAMERA:
            cv2.putText(image, 'left_motor: {:.2f}'.format(x), (10, 300), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, 'right_motor: {:.2f}'.format(y), (10, 350), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
     # 紀錄這一張圖片
    if DO_RECORD_CAMERA:
        out.write(change['new'])

    # 計算FPS
    fps = 1 / (time.time() - start)
#     print("\rFPS: {:.0f}".format(fps), end='')

camera.observe(execute, names='value')

Image(value=b'')

In [22]:
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
if DO_MOTOR:
    robot.stop()
camera.stop()